# Disk Failure Analysis
 * A preliminary analyis of hard drives 2016 from Backblaze. This dataset contains basic hard drive information and 90 columns of raw and normalized values of 45 different S.M.A.R.T. statistics. Each row represents a daily snapshot of one hard drive.
 * code source: https://github.com/bastidas/disk_analysis
 * data source: https://www.backblaze.com/b2/hard-drive-test-data.html
 * alexander.bastidas.fry@gmail.com

In [ ]:
import os
import sys
import math
import struct
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

dir = "/home/alex/Downloads/data_Q1_2016/"
hd = pd.DataFrame()
for file in os.listdir(dir): # loading 78 indivual data files
    temp = pd.read_csv(dir+file, header=0,nrows=20000) # my laptop is struggling to handle more rows
    hd = hd.append(temp)
print("Shape of hd data: ", np.shape(hd))
print("There are %d unique drives. " % hd['serial_number'].value_counts().count())
print("There are %d unique models. " % hd['model'].value_counts().count())
print("There are %d unique dates. " % hd['date'].value_counts().count())
print("There are %d failures." % hd['failure'].sum())

### Select hard drive models for which we have at least on failure.

In [ ]:
hd_models = hd.groupby("model").agg({"failure": np.sum, "serial_number": pd.Series.nunique})
hd_models_index = hd_models.index.tolist()
hd_failure_count = hd_models.failure.tolist()
n_required = 5 
for q in range(len(hd_models)):
    if hd_failure_count[q] < n_required:
        #print(hd_models_index[q])
        hd = hd[hd.model != hd_models_index[q]]
hd_models = hd.groupby("model").agg({"failure": np.sum, "serial_number": pd.Series.nunique})
print('hd models', hd_models)

In [ ]:
def tb_capacity(x):
    """
    1 gig is np.power(2, 30) bytes, but maybe it is 10^9 bytes. Who you asking? Whatever.
    """
    tb = np.power(10, 12)
    #if not math.isnan(x):
    return x/tb

hd.capacity_bytes = hd.capacity_bytes.map(tb_capacity)
hd.rename(columns={'capacity_bytes': 'capacity_tb'}, inplace=True)
print(hd.groupby('capacity_tb').size()) # looks like the capacity is stored irregularly

In [ ]:
hd.loc[:, 'date'] = pd.to_datetime(hd.loc[:, 'date'])
hd.loc[:, 'date'] = hd.loc[:, 'date'] .dt.dayofyear
hd.rename(columns={'date': 'day_of_year'}, inplace=True)

In [ ]:
models = hd['model'].value_counts().index.tolist()
hd_by_model =[]
aggregations = {
    'failure': {
     'failure': 'sum'
    },  
    'serial_number':{
         'unique_serial': pd.Series.nunique   
        }
}

for q in models: #for each model get its stats per day
    ags = hd[hd['model'] == q].groupby("day_of_year").agg(aggregations)#.reset_index()
    ags.columns = ags.columns.droplevel()
    hd_by_model.append(ags)

tot_failures = []    
for q in range(len(models)):
    tot_failures.append( np.cumsum(hd_by_model[q]['failure']))

In [ ]:
for n in range(len(hd_by_model)):
    failure_rate = 100*365*np.mean(hd_by_model[n].failure/hd_by_model[n].unique_serial)
    print("%s fails at an annual percentage rate of %s percent." % (models[n],np.round(failure_rate,1)))

In [ ]:

sn_colors = sns.color_palette("hls", 8)
tmark=""
fig = plt.figure(figsize=(1,1))
ax = fig.add_subplot(111)
for tcolor, umn in zip(sn_colors, range(len(models))):
    ax.plot([None], [None], marker=tmark, color=tcolor, lw=1.0,label = models[umn])
leg = plt.legend(loc='center', shadow=False, frameon=True, ncol=2, borderaxespad=0.)
leg.get_frame().set_linewidth(0.5)
plt.axis('off')
plt.show()

fig = plt.figure(figsize=(6,6))
fig = plt.figure()
ax = fig.add_subplot(221)
for tcolor, umn, in zip(sn_colors, range(len(models))):
    ax.plot(hd_by_model[umn].index, hd_by_model[umn].failure, marker=tmark, color=tcolor, lw=1.0)
ax.set_xlabel('Day')
ax.set_ylabel('# failures')
ax.set_xlim(0)
ax.set_ylim(0)
ax = fig.add_subplot(222)
for tcolor, umn in zip(sn_colors, range(len(models))):
    ax.semilogy(hd_by_model[umn].index, hd_by_model[umn].unique_serial, marker=tmark, color=tcolor, lw=1.0)
ax.set_xlabel('Day')
ax.set_ylabel('# of unique drives')
ax = fig.add_subplot(223)
for tcolor, umn in zip(sn_colors, range(len(models))):
    ax.plot(hd_by_model[umn].index, tot_failures[umn], marker=tmark, color=tcolor, lw=1.0)
ax.set_xlabel('Day')
ax.set_ylabel('cumulative # failures')
ax = fig.add_subplot(224)
for tcolor, umn in zip(sn_colors, range(len(models))):
    ax.plot(hd_by_model[umn].index, tot_failures[umn]/hd_by_model[umn].unique_serial, marker=tmark, color=tcolor, lw=1.0)
ax.set_xlabel('Day')
ax.set_ylabel('Cumulative Failure Fraction')
plt.tight_layout()
plt.show()


# What is the median survival time of a hard drive?
* Survival analysis for this, in particular the [lifelines package](http://lifelines.readthedocs.io/en/latest/). Note that this data is truncated and censored, what that means is that we need to use careful statistics to predict reasonable survival times and reasonable confidence levels of those predictions.

In [ ]:
model_map = dict(zip(hd_models.index.tolist(),range(len(hd_models)))) # map the model names to integers for convenience
hd = hd.replace({'model': model_map})
print(model_map)

In [ ]:
#seagates = pd.DataFrame.copy(hd[hd['model'] == 'ST4000DM000']) # selected the most common seagate model
aggregations = {
    'day_of_year': { 
        'first_date': 'min',  
        'last_date': 'max', 
        'uncensored_days': 'count',
        'days': lambda x: max(x) - min(x)  
    },
    'smart_9_raw': { # smart 9 is the disk uptime
        'runtime_max': 'max',  
        'runtime_min': 'min',
        'uptime': lambda x: max(x) - min(x)  
    },
    'model':{
       'model_count': 'count',
       'model': 'mean'
    },
    'failure': {
     'failure': 'sum'
    }  
}
survival = hd.groupby('serial_number').agg(aggregations).reset_index()
survival.columns = survival.columns.droplevel()

In [ ]:
survival = survival.loc[survival['failure'] <=1 ]
survival['runtime_max'] = survival['runtime_max']/8760.0 # hrs to days
survival['runtime_min'] = survival['runtime_min']/8760.0
print(survival['model'].value_counts())
del survival['model_count']

## Kaplan-Meier Modeling

 *    Kaplan-Meier is a non-parametric non-generalising maximum-likelihood estimate of the survival function, S(t), which is the probability of survival until time t.
 *   The data is left truncated: hard drives enter study interval at different times. So we use the 'entry' keyword (Did I interpret the lifelines documentation right?)
 *   If we had a parametric form of the survival function we could predict new unseen data and predict the relative impact on survival of various hard drive attributes...



In [ ]:
import lifelines as sa
km = sa.KaplanMeierFitter()
km.fit(durations=survival['runtime_max'], event_observed=survival['failure'], entry=survival['runtime_min'])
S = km.survival_function_
ax = km.plot(title="Kaplan Meier fit", legend=False)
ax.annotate('half-life', color='black', xy=(0.01,0.5), xycoords='axes fraction', xytext=(10,4), textcoords='offset points')
ax.axhline(.5, ls='--', lw=1.0, color='black')
ax.set_ylim([0,1])
#ax.set_xlim([0,6])
plt.show()

## Hazard rates with Nelson-Aalen
 * The failure rate is the total number of failures within a population, divided by the total time expended by that population, during a particular measurement interval.
 * The hazard function or hazard rate is the failure rate calculated instantaneously.  
 * The cumulative hazard curve is a basic tool: it is the sum of failure rate estimates so it is much more stable than the point-wise instananeous estimates.
 * The hazard curve has a catch: the derivation involves a smoothing kernel smoother applied to the differences of the cumulative hazard curve), and thus it has a free parameter.

In [ ]:
from lifelines import NelsonAalenFitter
naf = NelsonAalenFitter()
seagate = survival.loc[survival['model'] == model_map['ST4000DM000']]
naf.fit(seagate['runtime_max'], seagate['failure'], entry=seagate['runtime_min'], label='Seagate ST4000DM000')
fig, axes = plt.subplots(nrows=1, ncols=1, squeeze=False, sharex=True, sharey=True)
naf.plot(ax=axes[0,0],title="Cumulative Hazard Rate")
plt.show()

haz = naf.smoothed_hazard_
smoothing_bandwidth_time=1.0
q = haz(smoothing_bandwidth_time)
mean_haz = np.mean( q[q.columns[0]])

ax = naf.plot_hazard(bandwidth=smoothing_bandwidth_time, title = "Hazard Rate")
ax.axhline(mean_haz , ls='--', lw=1.0, color='black')
haz_str = str(100*np.round(mean_haz,4)) + "%"
ax.annotate(haz_str, color='black', xy=(0.01,mean_haz), xytext=(10,4), textcoords='offset points')
plt.show()
print("The failure rate for these seagate drives is %s" % haz_str)

### That was a lot of work to get basically the same failure rate that we calculated all the way back at the begining. Further survival regression analyis would be useful, but lets move on from survival analysis for now and ask another question.

##  Predict which hard drives will fail given the hard drive statistics?
 * So there isn't a lot of information about drive failures, but can we at least find what hard drive diagnostics (the smart values) are preddictive of a hard drive failure?
 * Most of the data columns are S.M.A.R.T. values that can vary in meaning based on the manufacturer and model. For this reason at this time limit the more in depth analysis to the most common identical drive model. 

In [ ]:
y = hd.failure

### Get just the raw columns and ignore columns with bad data.

In [ ]:
def clip_by_column_txt(df, col_txt):
    for ikey in df.keys():
        if col_txt not in ikey:
            del df[ikey]

clip_by_column_txt(hd,"raw") # select raw columns only
hd = hd.dropna(axis=1)
#sel = VarianceThreshold(threshold=.01)  # remove the features with variance below threshold.
#sel.fit_transform(hd)
#print(hd.head(3))

### There are a few ways you could decide to clip the important features... 

In [ ]:
# Use forest classifier to find important features
forest = ExtraTreesClassifier(n_estimators=250, random_state=0)
forest.fit(hd, y)
importance = forest.feature_importances_
std_err = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
# get the indices and sort the labels of the important features
indices = np.argsort(importance)[::-1]
x_labels = [x for (y, x) in sorted(zip(importance, hd.columns))][::-1]


### In the plot the dashed line is the median of the importance which is the chosen threshold.

In [ ]:
# clip the features to the most important ones
thresh = np.median(importance)
clip_importance = importance[np.where(importance >= thresh)]
clip_indices = indices[0:len(clip_importance)]
clip_x_labels = x_labels[0:len(clip_importance)]
for i in range(len(clip_indices)):
    print("%d. clipped feature %s (%f)" % (i + 1, clip_x_labels[i], importance[clip_indices[i]]))
sn_colors = sns.color_palette()
plt.figure()
plt.title("Feature Importances")
plt.bar(range(len(clip_indices)), importance[clip_indices], color=sn_colors[1],
        yerr=std_err[clip_indices], align="center")
# Plot the mean line, below using SelectFromModel we will clip features below this value
plt.axhline(np.median(importance), ls='--', lw=2.0, color='black', alpha=99)
plt.xticks(range(len(clip_indices)), clip_x_labels, rotation=80)
plt.xlim([-1, len(clip_indices)])
plt.ylim([0, .5])
plt.show()